# Using `opto.trainer` algorithms for scaling up generative optimization

This tutorial walks you through the different algorithms that have been built on top of the generative optimizers in Trace.
The `minibatch` tutorial already showed one specific use-case: `MiniBatchAlgorithm` that takes an agent, dataset and opto optimizer as inputs and outputs an optimized agent. 
In fact, all of the algorithms in `opto.trainer` obey this basic input-output mapping; they all use the opto optimizers to propose candidate parameters, but utilize different search procedures on top of that to refine the optimized agent.

We will use the [HardMath dataset](https://huggingface.co/datasets/xuanfeiren/math_hard_gemini) in this tutorial to illustrate the various algorithms in `opto.trainer`.

In [ ]:
%pip install trace-opt ipywidgets

The code below provides a way to specify your API_KEY for calling LLMs using LiteLLM as part of this tutorial notebook. Alternatively, provide the keys by setting environment variables or loading LiteLLM config files.

In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display

# Function to save the environment variable and API key
def save_env_variable(env_name, api_key):
    # Validate inputs
    if not env_name.strip():
        print("⚠️ Environment variable name cannot be empty.")
        return
    if not api_key.strip():
        print("⚠️ API key cannot be empty.")
        return
    
    # Store the API key as an environment variable
    os.environ[env_name] = api_key
    globals()[env_name] = api_key  # Set it as a global variable
    print(f"✅ API key has been set for environment variable: {env_name}")

# Create the input widgets
env_name_input = widgets.Text(
    value="OPENAI_API_KEY",  # Default value
    description="Env Name:",
    placeholder="Enter env variable name (e.g., MY_API_KEY)",
)

api_key_input = widgets.Password(
    description="API Key:",
    placeholder="Enter your API key",
)

# Create the button to submit the inputs
submit_button = widgets.Button(description="Set API Key")

# Display the widgets
display(env_name_input, api_key_input, submit_button)

# Callback function for the button click
def on_button_click(b):
    env_name = env_name_input.value
    api_key = api_key_input.value
    save_env_variable(env_name, api_key)

# Attach the callback to the button
submit_button.on_click(on_button_click)

We load the dataset and define a `Guide` (i.e. LLM-as-Judge) that can provide feedback for answers to questions in the dataset.

In [4]:
import datasets
import numpy as np
from typing import Any, Tuple
from opto.trainer.guide import AutoGuide
from opto.utils.llm import LLM

# Set random seed
np.random.seed(42)

math_data = datasets.load_dataset('xuanfeiren/math_hard_gemini')
train_data = math_data['train'].select(
        range(10, 30)
    )
validate_data = train_data
test_data = math_data['test'].select(range(10))

# Format data for trainer
train_dataset = {'inputs': train_data['problem'], 'infos': train_data['solution']}
validate_dataset = {'inputs': validate_data['problem'], 'infos': validate_data['solution']}
test_dataset = {'inputs': test_data['problem'], 'infos': test_data['solution']}

# Log dataset sizes
print(f"Training samples: {len(train_dataset['inputs'])}")
print(f"Validation samples: {len(validate_dataset['inputs'])}")
print(f"Test samples: {len(test_dataset['inputs'])}")


class TeacherGuide(AutoGuide):
    """Guide that uses LLM to judge answers and provide feedback."""
    
    def __init__(self, model: str = "gpt-4o-mini"):
        """Initialize the teacher guide.
        
        Args:
            model: The LLM model to use for evaluation
        """
        super().__init__()
        self.guide_llm = LLM(model=model)
        self.system_prompt = "You are an expert math teacher evaluating student answers."
        self.judge_prompt_template = (
            "Carefully review the following three distinct sections:\n\n"
            "SECTION 1: The Math Problem\n"
            "----------------------------\n"
            "{query}\n"
            "----------------------------\n\n"
            "SECTION 2: The Student's Full Answer\n"
            "----------------------------\n"
            "{response}\n"
            "----------------------------\n\n"
            "SECTION 3: The Official Correct Answer\n"
            "----------------------------\n"
            "{reference}\n"
            "----------------------------\n\n"
            "INSTRUCTIONS FOR JUDGING:\n"
            "1. Your primary task is to compare the student's **final numerical result** (or final conclusion if no number is present) from SECTION 2 with the **Official Correct Answer** provided in SECTION 3.\n"
            "2. When evaluating SECTION 2 (Student's Full Answer), focus SOLELY on the **final answer part** of the student's response. Ignore all intermediate steps, reasoning, or explanations for the correctness check unless the problem specifically asks for reasoning as the final answer.\n"
            "3. Determine if the student's **final answer** is equivalent to the **Official Correct Answer**.\n\n"
            "RESPONSE FORMAT:\n"
            "- If the student's final answer (from SECTION 2) IS equivalent to the Official Correct Answer (from SECTION 3), respond ONLY with the exact phrase: 'Correct [TERMINATE]'\n"
            "- If the student's final answer IS NOT equivalent, respond ONLY with specific and actionable feedback. The feedback should clearly explain the error in the student's final answer and guide them on how to arrive at the Official Correct Answer."
        )

    def get_feedback(self, task: str, response: str, info: Any, **kwargs) -> Tuple[float, str]:
        """Get feedback on a student response.
        
        Args:
            task: The original math problem
            response: The student's answer
            info: The reference/correct answer
            **kwargs: Additional arguments
            
        Returns:
            Tuple of (score, feedback_text)
        """
        user_prompt = self.judge_prompt_template.format(
            query=task,
            response=response,
            reference=info
        )

        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": user_prompt}
        ]

        llm_response = self.guide_llm(messages=messages)
        feedback_text = llm_response.choices[0].message.content

        if 'Correct [TERMINATE]' in feedback_text:
            return 1.0, "Correct."
        else:
            return 0.0, f"Incorrect. Feedback: {feedback_text}"
    
    def metric(self, task: str, content: str, info: Any, **kwargs) -> float:
        """Calculate the metric score for an answer.
        
        Args:
            task: The original math problem
            content: The student's answer
            info: The reference/correct answer
            **kwargs: Additional arguments
            
        Returns:
            Score (0.0 or 1.0)
        """
        score, _ = self.get_feedback(task, content, info, **kwargs)
        return score

/home/aswaminathan/miniconda3/envs/trace/lib/python3.9/site-packages/flaml/__init__.py:20: UserWarning: flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.
  warnings.warn("flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.")


Training samples: 20
Validation samples: 20
Test samples: 10


We define the `Learner` agent which is a student LLM with a trainable `system prompt` and a trainable `user prompt template`. Trace will use a generative optimizer to tune these prompts.

In [5]:
from opto import trace
from opto.optimizers import OptoPrime
from opto.optimizers.utils import print_color
from opto.trace.modules import Module
from opto.trainer.algorithms.basic_algorithms import MinibatchAlgorithm, BasicSearchAlgorithm
from opto.trainer.algorithms.beamsearch_algorithm import BeamsearchAlgorithm, BeamsearchHistoryAlgorithm
from opto.trainer.algorithms.UCBsearch import UCBSearchAlgorithm


@trace.model
class Learner(Module):
    """A basic LLM Agent for solving math problems."""
    
    def __init__(self, 
                system_prompt: str = "You're a helpful agent answering math problems.",
                user_prompt_template: str = "Solve the following math problem step-by-step: {message}",
                llm: LLM = None):
        """Initialize the learner agent.
        
        Args:
            system_prompt: System prompt to guide LLM behavior
            user_prompt_template: Template for formatting user messages
            llm: LLM instance to use for generation (defaults to gpt-3.5-turbo)
        """
        super().__init__()
        self.system_prompt = trace.node(system_prompt, trainable=True)
        self.user_prompt_template = trace.node(user_prompt_template, trainable=True)
        self.llm = llm or LLM(model="gpt-3.5-turbo")

    @trace.bundle()
    def call_llm(self, system_prompt: str, user_prompt: str) -> str:
        """Call LLM model with the given prompts.
        
        Args:
            system_prompt: The system prompt
            user_prompt: The user prompt
            
        Returns:
            The LLM response content
        """
        response = self.llm(
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )
        return response.choices[0].message.content

    def forward(self, message: Any) -> str:
        """Agent's forward pass to process a message.
        
        Args:
            message: The input message to process
            
        Returns:
            The generated response
        """ 
        user_prompt = self.user_prompt_template.format(message=message)
        return self.call_llm(self.system_prompt, user_prompt)


We initialize all the components: the agent using the student LLM, the guide using the teacher LLM, and the optimizer using an LLM as a generative optimizer.

In [6]:
student_llm = LLM()
agent = Learner(llm=student_llm)

train_guide = TeacherGuide()
validate_guide = TeacherGuide()

optimizer = OptoPrime(agent.parameters())

from opto.trainer.loggers import DefaultLogger
class SimpleLogger(DefaultLogger):
    """Simplified logger that only shows important metrics."""
    
    def log(self, name: str, data: Any, step: int, **kwargs):
        """Log only specific metrics to reduce output clutter.
        
        Args:
            name: The name of the metric
            data: The metric value
            step: The current step
            **kwargs: Additional logging arguments
        """
        important_metrics = [
            'Average train score',
            'Average test score',
            'Validation score'
        ]
        
        if name in important_metrics or 'Parameter' in name:
            super().log(name, data, step, **kwargs)

logger = SimpleLogger()

import nest_asyncio
nest_asyncio.apply()
import asyncio

train_params = {
        "guide": train_guide,
        "train_dataset": train_dataset,
        "num_epochs": 1,
        "num_threads": 5,
        "batch_size": 5,
        "test_dataset": test_dataset,
        "validate_dataset": validate_dataset,
        "validate_guide": validate_guide,
        "test_frequency": 2,
        "log_frequency": 2,
        #for Basic Search
        "num_proposals": 2,
        #for Beam Search
        "validation_dataset_size": 5,
        "beam_width": 3,
        "max_depth": 4,
        "max_history_size": 2,
        #for UCB Search
        "num_search_iterations": 3,
        "train_batch_size": 5,
        "evaluation_batch_size": 5,
        "max_buffer_size": 3,
        "ucb_exploration_factor": 1.0
    }

Finally, we will go through each of the algorithms in `opto.trainer`. Each algorithm will run the student model on the train dataset, gather feedback from the teacher model, present the resulting traced graph to the optimizer, and then perform specific post-processing throughout each training epoch.

In [7]:
algorithm = MinibatchAlgorithm(
            agent=agent,
            optimizer=optimizer,
            logger=logger,
            num_threads=train_params["num_threads"]
        )

async def wrapper():
    print("STARTING TRAINING MINIBATCH")
    metrics, final_score = algorithm.train(**train_params)
    print("FINISHED TRAINING MINIBATCH")
    print("Final score: ", final_score)

asyncio.run(wrapper())

STARTING TRAINING MINIBATCH


Evaluating agent (iteration 0): 100%|██████████| 10/10 [00:52<00:00,  5.26s/it]


[Step 0] Average test score: 0.4


Evaluating agent (iteration 2): 100%|██████████| 10/10 [00:50<00:00,  5.06s/it]


[Step 2] Average test score: 0.2
Epoch: 0. Iteration: 2
[Step 2] Average train score: 0.2
[Step 2] Parameter: str:0: You're a helpful agent assisting with thorough and complete mathematical problem analysis, ensuring all steps are accurately validated.
[Step 2] Parameter: str:1: Carefully process each subcomponent of the following problem: {message} Methodically ensure completeness in probability calculations, permutations, customizable solutions, and systematic explorations of possible outcomes.


Evaluating agent (iteration 4): 100%|██████████| 10/10 [01:01<00:00,  6.10s/it]

[Step 4] Average test score: 0.2
Epoch: 0. Iteration: 4
[Step 4] Average train score: 0.2
[Step 4] Parameter: str:0: Accurate precision ensuring number coating and span impart cataloguing upon probability, permutation, solution synthesis, and structured exploration
[Step 4] Parameter: str:1: Diligently analyze each part facet of the offering issue: {message} carefuly ascertain completion in probability computation, permutation exercise, customizable provides solution, and scheme sized explorable outcomes.
FINISHED TRAINING MINIBATCH
Final score:  0.2


In [8]:
algorithm = BasicSearchAlgorithm(
            agent=agent,
            optimizer=optimizer,
            logger=logger,
            num_threads=train_params["num_threads"]
        )

async def wrapper():
    print("STARTING TRAINING BASIC SEARCH")
    metrics, final_score = algorithm.train(**train_params)
    print("FINISHED TRAINING BASIC SEARCH")
    print("Final score: ", final_score)
    
asyncio.run(wrapper())

STARTING TRAINING BASIC SEARCH


Evaluating agent (iteration 0): 100%|██████████| 10/10 [01:06<00:00,  6.63s/it]


[Step 0] Average test score: 0.2


Validating proposals: 100%|██████████| 20/20 [02:16<00:00,  6.82s/it]


[Step 0] Validation score: 0.05


...

[Step 2] Validation score: 0.15


Validating proposals: 100%|██████████| 20/20 [01:25<00:00,  4.25s/it]


[Step 3] Validation score: 0.15


Evaluating agent (iteration 4): 100%|██████████| 10/10 [00:45<00:00,  4.52s/it]

[Step 4] Average test score: 0.3
Epoch: 0. Iteration: 4
[Step 4] Average train score: 0.15000000000000002
[Step 4] Parameter: str:0: Critically examine and describe each step of the problem-solving process, ensuring thorough precision in applying combinatorial logic, sequence conversions, and probability distributions within complex scenarios such as probability computation, permutation exercise, solution synthesis, and exploration of structured outcomes.
[Step 4] Parameter: str:1: Evaluate each component in detail for the given problem situation: {message} employing strategic reasoning to ascertain completion in logical computation, solving exercises through permutations, offering customizable solutions, and unveiling outcomes of scenario explorations.
FINISHED TRAINING BASIC SEARCH
Final score:  0.3


In [9]:
algorithm = BeamsearchAlgorithm(
            agent=agent,
            optimizer=optimizer,
            logger=logger,
            num_threads=train_params["num_threads"]
        )

async def wrapper():
    print("STARTING TRAINING BEAM SEARCH")
    metrics, final_score = algorithm.train(**train_params)
    print("FINISHED TRAINING BEAM SEARCH")

    if 'best_validation_scores' in metrics:
        print("\nBest validation scores at each depth:")
        for depth, score in enumerate(metrics['best_validation_scores']):
            print(f"  Depth {depth+1}: {score:.4f}")
            
    print("Final score: ", final_score)
    
asyncio.run(wrapper())

STARTING TRAINING BEAM SEARCH
Running BeamsearchAlgorithm with beam_width=3, max_depth=4
Using validation_dataset_size=5 for intermediate evaluations

===== Evaluating Initial Parameters =====


Evaluating initial parameters on test set: 100%|██████████| 10/10 [00:41<00:00,  4.18s/it]


Initial test score: 0.2000

===== Beam Search Depth 1/4 with 1 beams =====
Sampled validation minibatch of size 5 for depth 1
Processing beam 1/1


Generating 2 proposals for beam 1:  50%|█████     | 1/2 [00:09<00:09,  9.32s/it]

LLM response:
 {
    "reasoning": "The feedback provided indicates issues with the outcomes computed in the code for some problem instances. Here's a breakdown:\n1. ID[0]: The student's calculated answer was off due to an incorrect count of distinct collections of consonants. They provided 87 when the correct count is 72. This suggests re-evaluating how the consonants are grouped without double-counting. The construction of possible usage scenarios needs correction to prevent overlap and ensure unique contributions.\n2. ID[1] was correct, so no changes are needed for this problem.\n3. ID[2]: The student's understanding of permutations and probabilities based on the lattice was incorrect. They concluded with a probability of 1/16, but the correct symmetry of movements on the lattice results in a probability of 1/4. This indicates a need to consider the even distribution across potential endpoints on the lattice, using symmetry to realize each endpoint is equally probable.\n4. ID[3] was 

...

Candidate 2: Validation score: 0.0000


Validating candidate 3/3: 100%|██████████| 20/20 [02:31<00:00,  7.58s/it]


Candidate 3: Validation score: 0.0500
Selected top 1 beams with scores: ['0.0500']

===== Final Proposal Candidate Parameters =====
str:0: Solve each problem by directly calculating distinct answers. For the object reaching (2,2) in steps, determine all ways in 4 or 6 steps. For the lockers, trace each open-close step carefully until locker 342 is the last. For gymnasts, calculate combinations where total is precisely 281, minimizing coach handshakes. For cousins, enumerate placement variants for each room combination. For letters, determine indistinguishable combinations focusing on T's, M's, and A's.
str:1: Focus directly on providing numeric answers by applying combinatorics, symmetry, and dimensions fitting logic specific to the distinct problem being queried, ensuring detailed, step-by-step solutions addressing the exact scenarios described.


Evaluating best beam on test set: 100%|██████████| 10/10 [00:54<00:00,  5.48s/it]

BEST BEAM - Test score: 0.0000

===== Periodic Test Scores Summary =====
Depth 1: Test score = 0.2000
Depth 4: Test score = 0.0000
FINISHED TRAINING BEAM SEARCH

Best validation scores at each depth:
  Depth 1: 0.6000
  Depth 2: 0.0000
  Depth 3: 0.2000
  Depth 4: 0.2000
Final score:  0.0


In [10]:
algorithm = BeamsearchHistoryAlgorithm(
            agent=agent,
            optimizer=optimizer,
            logger=logger,
            num_threads=train_params["num_threads"]
        )

async def wrapper():
    print("STARTING TRAINING BEAM SEARCH w/ HISTORY")
    metrics, final_score = algorithm.train(**train_params)
    print("FINISHED TRAINING BEAM SEARCH w/ HISTORY")

    if 'best_validation_scores' in metrics:
        print("\nBest validation scores at each depth:")
        for depth, score in enumerate(metrics['best_validation_scores']):
            print(f"  Depth {depth+1}: {score:.4f}")
            
    print("Final score: ", final_score)
    
asyncio.run(wrapper())

STARTING TRAINING BEAM SEARCH w/ HISTORY
Running BeamsearchHistoryAlgorithm with beam_width=3, max_depth=4, max_history_size=2
Using validation_dataset_size=5 for intermediate evaluations

===== Evaluating Initial Parameters =====


Evaluating initial parameters on test set: 100%|██████████| 10/10 [00:59<00:00,  5.95s/it]


Initial test score: 0.0000

===== Beam Search Depth 1/4 with 1 beams =====
Sampled validation minibatch of size 5 for depth 1
Processing beam 1/1


Generating 2 proposals for beam 1 (with history): 100%|██████████| 2/2 [00:18<00:00,  9.20s/it]


LLM response:
 {
"reasoning": "1. The instruction requires modifying the values of the variables in #Variables to improve the output. 2. Based on the feedback, it is evident that the calculations or expected outputs do not match the Official Correct Answer. Many of the provided answers do not align with the expected numbers such as m+n = 67, which appears to relate to reaching (2,2) in six or fewer steps in a given problem (assuming the task requests this directly by formula derivation), probability of being at dot B is 1/4 for the ant problem, 72 for the collection of letters problem, 560 for the sequence problem, and 336 for the distinguishable triangles. All these need revisiting. 3. The feedback indicates each computation feature problems deviating from core constraints needed or crossing excess assumptions weakening finite numerical submission. 4. Suggestion: I suggest refining str0 and str1 further to focus on precise calculation statements addressing each scenario uniquely but c

...

Candidate 2: Validation score: 0.0000


Validating candidate 3/3: 100%|██████████| 20/20 [00:45<00:00,  2.26s/it]


Candidate 3: Validation score: 0.0000
Selected top 1 beams with scores: ['0.1500']

===== Final Proposal Candidate Parameters =====


Evaluating best beam on test set: 100%|██████████| 10/10 [00:48<00:00,  4.81s/it]

BEST BEAM - Test score: 0.3000

===== Periodic Test Scores Summary =====
Depth 1: Test score = 0.0000
FINISHED TRAINING BEAM SEARCH w/ HISTORY

Best validation scores at each depth:
  Depth 1: 0.0000
  Depth 2: 0.2000
  Depth 3: 0.2000
  Depth 4: 0.2000
Final score:  0.3


In [11]:
algorithm = UCBSearchAlgorithm(
            agent=agent,
            optimizer=optimizer,
            logger=logger,
            num_threads=train_params["num_threads"],
            max_buffer_size=train_params["max_buffer_size"],
            ucb_exploration_factor=train_params["ucb_exploration_factor"]
        )

async def wrapper():
    print("STARTING TRAINING UCB SEARCH")
    metrics, final_score = algorithm.train(**train_params)
    print("FINISHED TRAINING UCB SEARCH")

    if 'best_candidate_scores' in metrics and metrics['best_candidate_scores']:
        print(f"  Best candidate scores over iterations: {len(metrics['best_candidate_scores'])} recorded")
        print(f"  Final best candidate score: {metrics['best_candidate_scores'][-1]:.4f}")
    if 'buffer_avg_score' in metrics and metrics['buffer_avg_score']:
        print(f"  Final buffer average score: {metrics['buffer_avg_score'][-1]:.4f}")
            
    print("Final score: ", final_score)
    
asyncio.run(wrapper())

STARTING TRAINING UCB SEARCH
Evaluating initial parameters using validation_dataset samples...


Evaluating candidate: 100%|██████████| 5/5 [00:32<00:00,  6.47s/it]


Initial candidate: Score 0.2000, Evals 5
Iter 1/3: 


Iter 1: Forward pass for action 'a' : 100%|██████████| 5/5 [00:24<00:00,  4.95s/it]


LLM response:
 {
    "reasoning": "The feedback points out errors in the calculations for each task performed by the code. The main issue across the tasks is an incorrect approach or missed key calculations that lead to incorrect results. For the sequences of coin tosses problem, the student failed to properly account for combinatorial arrangements using the 'balls and urns' model, leading to a severely inflated number of possible sequences. Similarly, for the fish population problem, the proportions were not used correctly to derive the number of fish, resulting in a projection error in the population. In the locker problem, improper tracking of the opening and closing pattern led to identifying the wrong last locker number. The card order problem had overcounting issues because of incorrectly accounting for overcounted sequences due to adjacent swaps. Lastly, the tower height estimation miscalculated possible heights due to incorrect accounting for achievable combinations. To tackle 

...

Iter 2: New candidate a_prime generated. Validation Score: 0.0000, Evals: 5
Iter 2: Added new candidate to buffer.
Log @ Iter 2: Best score in buffer: 0.1000, Buffer size: 3, Total samples: 25
Iter 3/3: 


Iter 3: Forward pass for action 'a' : 100%|██████████| 5/5 [00:33<00:00,  6.74s/it]


LLM response:
 {
    "reasoning": "1. The #Instruction requires a focus on improving the quality of outputs by changing #Variables values in accordance with #Feedback. 2. The #Feedback indicates that the outputs do not address the specific problem requirements, focusing instead on unrelated subjects like domino patterns or complex methodologies. The outputs need to connect directly with the core mathematical problems presented in #Inputs. 3. Each of the five problems presented requires distinct solutions: a probability calculation, a combinatorial typing sequence, a gymnastics handshake count, a card arrangement for sequences, and a triangle arrangement problem. Each requires a direct numerical solution. 4. Given the instruction, the value of 'str1' should help the code focus explicitly on the combinatorial problems at hand and ensure proper filling of user prompts according to input requirements. To remedy this, the prompt should directly respond to the particular problems' constraint

Evaluating candidate: 100%|██████████| 5/5 [00:06<00:00,  1.25s/it]

Iter 3: New candidate a_prime generated. Validation Score: 0.0000, Evals: 5
Iter 3: Buffer full. Evicted a candidate (UCB: 0.5963)
Iter 3: Added new candidate to buffer.
UCB search finished.
Final best candidate: Mean Score 0.1000, Evals 10
FINISHED TRAINING UCB SEARCH
  Best candidate scores over iterations: 3 recorded
  Final best candidate score: 0.1000
  Final buffer average score: 0.0333
Final score:  0.1
